# Information Retrieval

For this type of system we will carry out a joint experimentation. Where we will experiment with:
- Text splitting
- Vectorisation method

To measure the quality of the system, we will use the defined metric TPPI.

## 0. Set up

In [ ]:
# Switch to home project directory
%cd ../..

In [ ]:
import json
import os

import pandas as pd
import numpy as np

import spacy

import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, spearmanr, kendalltau

from bert_score import BERTScorer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import logging
logging.set_verbosity_error()

from tqdm.notebook import tqdm

from textstat import flesch_reading_ease

from src.unite_talking_points.utils.config.config_loader import ConfigLoader
from typing import TypedDict
import openai

In [ ]:
config = ConfigLoader().load_config(current_directory_is_root=True)

In [ ]:
os.environ["OPENAI_API_KEY"] = config['External-services']['openai_api_key']
client = openai.OpenAI(api_key=config['External-services']['openai_api_key'])

In [ ]:
class TPPIResult(TypedDict):
    TPPI: float
    BertScore: float
    Normalized_BertScore: float
    Perplexity: float
    Normalized_Perplexity: float
    Flesch: float
    Normalized_Flesch: float


class TPPI:
    def __init__(self, model_type: str = 'bert-base-uncased'):
        """
        Inicializa el normalizador de puntuaciones y los modelos necesarios para los cálculos.
        """
        self.scorer = BERTScorer(model_type=model_type)
        self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    @staticmethod
    def normalize_bert_score(bert_score: float) -> float:
        """
        Normaliza el puntaje BERT en un rango de 0 a 1.
        """
        return (bert_score + 1) / 2

    @staticmethod
    def normalize_perplexity(perplexity: float) -> float:
        """
        Normaliza el valor de la perplejidad en un rango de 0 a 1.
        """
        # Aseguramos que la perplexidad mínima sea al menos 1 para evitar división por cero
        perplexity = max(perplexity, 1)
        # Invertimos la fórmula para que valores bajos de perplexidad den valores altos normalizados
        return 1 - (min(perplexity, 100) - 1) / 99

    @staticmethod
    def normalize_flesch(flesch: float) -> float:
        """
        Normaliza el puntaje de Flesch en un rango de 0 a 1.
        """
        return max(min(flesch, 100), 0) / 100

    def calculate_bert_score(self, reference_text: str, generated_text: str) -> float:
        """
        Calcula el BertScore entre un texto de referencia y uno generado.
        """
        P, R, F1 = self.scorer.score([reference_text], [generated_text], verbose=False)
        bert_score = F1.item()
        return bert_score

    def calculate_perplexity(self, text: str) -> float:
        """
        Calcula la perplejidad de un texto utilizando el modelo GPT-2.
        """
        encode = self.gpt2_tokenizer.encode(text, return_tensors='pt')
        with torch.no_grad():
            loss = self.gpt2_model(encode, labels=encode)[0]
        perplexity = torch.exp(loss).item()
        return perplexity

    @staticmethod
    def calculate_flesch(text: str) -> float:
        """
        Calcula el puntaje de Flesch Reading Ease de un texto.
        """
        flesch_score = flesch_reading_ease(text)
        return flesch_score

    def calculate_tppi(self, original_text: str, generated_text: str) -> TPPIResult:
        """
        Calcula el TPPI basado en los textos de referencia y generados, devolviendo los puntajes de BERT,
        perplejidad y Flesch tanto normalizados como sin normalizar.
        """
        # Calcula BertScore
        bert_score = self.calculate_bert_score(original_text, generated_text)
        normalized_bert_score = self.normalize_bert_score(bert_score)

        # Calcula Perplejidad
        perplexity = self.calculate_perplexity(generated_text)
        normalized_perplexity = self.normalize_perplexity(perplexity)

        # Calcula Flesch
        flesch = self.calculate_flesch(generated_text)
        normalized_flesch = self.normalize_flesch(flesch)

        # Calcula TPPI
        tppi_score = 0.5 * normalized_bert_score + 0.25 * normalized_perplexity + 0.25 * normalized_flesch

        # Devuelve todos los valores en un TypedDict
        return TPPIResult(
            TPPI=tppi_score,
            BertScore=bert_score,
            Normalized_BertScore=normalized_bert_score,
            Perplexity=perplexity,
            Normalized_Perplexity=normalized_perplexity,
            Flesch=flesch,
            Normalized_Flesch=normalized_flesch
        )


## 1. Data loading

In [ ]:
def load_json_data(data_dir):
    """
    Load JSON data from the specified directory into a pandas DataFrame.

    Args:
    - data_dir (str): Path to the directory containing JSON files.

    Returns:
    - df (pd.DataFrame): DataFrame containing the loaded JSON data.
    """
    # Initialize empty lists to store data
    file_names = []
    labels = []
    document_names = []
    meeting_names = []
    meeting_dates = []
    contents = []
    prompts = []

    # Iterate over each JSON file in the directory
    for filename in os.listdir(data_dir):
        if filename.endswith('.json'):
            with open(os.path.join(data_dir, filename), 'r') as file:
                data = json.load(file)
                # Extract data from each JSON file and append to lists
                file_names.append(filename)
                labels.append(data['label'])
                document_names.append(data['document_name'])
                meeting_names.append(data['meeting_name'])
                meeting_dates.append(data['meeting_date'])
                contents.append(data['content'])
                prompts.append(data['prompt'])

    # Create a DataFrame from the lists
    df = pd.DataFrame({
        'file_name': file_names,
        'label': labels,
        'document_name': document_names,
        'meeting_name': meeting_names,
        'meeting_date': meeting_dates,
        'content': contents,
        'prompt': prompts
    })

    return df

In [ ]:
df = load_json_data(config['Directories']['raw_data_path'])

In [ ]:
df

## 2. Data preprocessing

In [ ]:
# Load the English language model
nlp = spacy.load('en_core_web_trf')

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    # Parse the text with spaCy
    doc = nlp(text)
    
    # Tokenization, removing stop words, punctuation, and lemmatization
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    
    # Join the tokens back into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [ ]:
# Apply the preprocessing function to the 'content' column
df['preprocessed_content'] = df['content'].apply(preprocess_text)
df['preprocessed_prompt'] = df['prompt'].apply(preprocess_text)

# Display the preprocessed content
print("Preprocessed Content:")
print(df['preprocessed_content'].head())

In [ ]:
df['meeting_date'] = pd.to_datetime(df['meeting_date'], format='%d-%m-%Y')

## 3. Experimentation

### 3.1 Vectorization experimentation

In [ ]:
def get_openai_embedding(text, model="text-embedding-3-large"):
    response = client.embeddings.create(
        model=model,
        input=[text],
        encoding_format="float"
    )
    # Extract the embedding vector
    embedding = np.array(response.data[0].embedding).reshape(1, -1)
    return embedding

In [ ]:
# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['preprocessed_content'])
tfidf_prompt_matrix = tfidf_vectorizer.transform(df['preprocessed_prompt'])

# Vectorización con OpenAI
embedding_matrix = np.vstack(df['content'].apply(lambda x: get_openai_embedding(x)))
embedding_prompt_matrix = np.vstack(df['prompt'].apply(lambda x: get_openai_embedding(x)))

In [ ]:
# Initialize TPPI scoring system
tppi = TPPI()

# Function to conduct experiments
def conduct_experiment(df, vector_matrix, prompt_matrix, method):
    results = []
    for i in tqdm(range(len(df))):
        # Find the most relevant document
        cosine_similarities = cosine_similarity(prompt_matrix[i].reshape(1, -1), vector_matrix).flatten()
        cosine_similarities[i] = -1  # exclude the document itself
        most_relevant_doc_index = cosine_similarities.argmax()
        most_relevant_doc = df.iloc[most_relevant_doc_index]

        # Calculate TPPI
        tppi_result = tppi.calculate_tppi(df['content'][i], most_relevant_doc['content'])
        tppi_result['file_name'] = df['file_name'][i]
        tppi_result['method'] = method
        results.append(tppi_result)

    return results

# Run experiments
tfidf_results = conduct_experiment(df, tfidf_matrix, tfidf_prompt_matrix, "TF-IDF")
embedding_results = conduct_experiment(df, embedding_matrix, embedding_prompt_matrix, "Embedding")

# Combine and save results
all_results = pd.DataFrame(tfidf_results + embedding_results)

In [ ]:
# Calcula estadísticas básicas para cada método
statistics = all_results.groupby('method').agg({
    'TPPI': ['mean', 'std', 'min', 'max'],
    'BertScore': ['mean'],
    'Perplexity': ['mean'],
    'Flesch' : ['mean']
}).reset_index()

print(statistics)

In [ ]:
# Definir los nombres de las métricas para iterar
metrics = ['TPPI', 'Normalized_BertScore', 'Normalized_Perplexity', 'Normalized_Flesch']

# Crear un subplot 2x2 para los boxplots de cada métrica
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
fig.suptitle('Distribución de Métricas por Método de Vectorización', fontsize=16)

# Iterar sobre las métricas y los ejes para crear los boxplots
for ax, metric in zip(axes.flatten(), metrics):
    all_results.boxplot(column=metric, by='method', ax=ax, grid=False)
    ax.set_title(metric)
    ax.set_xlabel('Método de Vectorización')
    ax.set_ylabel('Puntuación')
    ax.set_ylim(0, 1)  # Asegurar que el gráfico esté limitado entre 0 y 1
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)  # Rotar las etiquetas para mejor legibilidad

# Ajustar el layout para evitar la superposición de los títulos y etiquetas
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Ajusta los límites del rectángulo del layout
plt.show()

### 3.2 Chuking experimentation

In [ ]:
def expand_by_newline(df):
    # Crear una lista de tuplas (índice original, frase)
    tuples = []
    for index, row in df.iterrows():
        for sentence in row['content'].split('\n'):
            tuples.append((index, row['file_name'], sentence.strip()))

    # Convertir la lista de tuplas en un DataFrame
    new_df = pd.DataFrame(tuples, columns=['original_index', 'file_name', 'sentence'])
    return new_df

In [ ]:
sentences = expand_by_newline(df)

In [ ]:
embedding_sentences_matrix = np.vstack(sentences['sentence'].apply(lambda x: get_openai_embedding(x)))
embedding_prompt_matrix = np.vstack(df['prompt'].apply(lambda x: get_openai_embedding(x)))

In [ ]:
def conduct_experiment(df, sentences, vector_matrix, prompt_matrix, n=5):
    results = []
    for i in tqdm(range(len(df))):
        # Find the most relevant document
        cosine_similarities = cosine_similarity(prompt_matrix[i].reshape(1, -1), vector_matrix).flatten()
        cosine_similarities[sentences.original_index == i] = -1  # exclude the document itself
        top_n_indices = cosine_similarities.argsort()[-n:][::-1]
        combined_content = '. '.join(sentences['sentence'][j] for j in top_n_indices)

        # Calculate TPPI
        tppi_result = tppi.calculate_tppi(df['content'][i], combined_content)
        tppi_result['file_name'] = df['file_name'][i]
        tppi_result['n'] = n
        results.append(tppi_result)

    return results

In [ ]:
all_results = []
for i in range(1, 11):
    all_results += conduct_experiment(df, sentences, embedding_sentences_matrix, embedding_prompt_matrix, i)

In [ ]:
all_results = pd.DataFrame(all_results)

In [ ]:
# Calcula estadísticas básicas para cada método
statistics = all_results.groupby('n').agg({
    'TPPI': ['mean', 'std', 'min', 'max'],
    'BertScore': ['mean'],
    'Perplexity': ['mean'],
    'Flesch' : ['mean']
}).reset_index()

print(statistics)

In [ ]:
# Calculating mean values for each 'n'
mean_values = all_results[['TPPI', 'Normalized_BertScore', 'Normalized_Perplexity', 'Normalized_Flesch', 'n']].groupby('n').mean().reset_index()

# Plotting
plt.figure(figsize=(12, 8))
plt.plot(mean_values['n'], mean_values['TPPI'], marker='o', label='Mean TPPI')
plt.plot(mean_values['n'], mean_values['Normalized_BertScore'], marker='o', label='Mean Normalized BertScore')
plt.plot(mean_values['n'], mean_values['Normalized_Perplexity'], marker='o', label='Mean Normalized Perplexity')
plt.plot(mean_values['n'], mean_values['Normalized_Flesch'], marker='o', label='Mean Normalized Flesch')

plt.title('Mean Evolution of TPPI and Normalized Scores by n')
plt.xlabel('Number of Combined Documents (n)')
plt.ylabel('Mean Score')
plt.xticks(mean_values['n'])  # Ensure x-axis labels match the number of documents
plt.legend()
plt.grid(True)
plt.show()